# PySpark Example of Machine Learning. 

## Importing the Data.  

In [1]:
# Importing Spark session. 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
# Read the Dataset.
training = spark.read.csv('test1.csv', header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [3]:
# Printing the Schema. 
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
# Printing the Columns.
training.columns

['Name', 'age', 'Experience', 'Salary']

### Vector Assembler.

In [5]:
# Importing the Vector Assembler. 
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol="Independent Features")

In [6]:
output = featureassembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [7]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [8]:
finalized_data = output.select('Independent Features','Salary')
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



## Training the Model.

In [14]:
from pyspark.ml.regression import LinearRegression
[train_data, test_data] = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [15]:
regressor.coefficients

DenseVector([83.8068, 1443.1818])

In [16]:
regressor.intercept

11796.874999999998

In [17]:
pred_result = regressor.evaluate(test_data)
pred_result.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|16610.795454545456|
|          [24.0,3.0]| 20000|18137.784090909092|
|          [30.0,8.0]| 25000| 25856.53409090908|
|         [31.0,10.0]| 30000|28826.704545454537|
+--------------------+------+------------------+



In [18]:
pred_result.meanAbsoluteError, pred_result.meanSquaredError

(1320.312499999999, 1877002.5584323339)